# Building on Slack

> Prerequisites: Complete Notebooks 1-3 and have your Slack App credentials ready. If you haven't set up your Slack App, follow the guide in the main README.md.

In our previous notebook, we built a webhook receiver to get real-time notifications from Manus. Now, we'll build on that foundation to create a bridge between Slack and Manus. This server will listen for events from Slack (like a user mentioning our bot) and translate them into tasks for Manus to execute.

Automations become truly useful when they react to real-world events as they happen. For many teams, the hub of activity is Slack—it's where support requests are flagged, bugs are reported, and ideas are discussed.

This is the crucial step before our final capstone project. We are building the "front door" for user interactions.

What You'll Learn in This Notebook

1. **Connecting to the Slack API**: You'll validate your environment credentials and learn the fundamentals of Slack's event-driven architecture, including how to securely verify incoming requests.
2. **Building a Slack Event Receiver**: We will deploy a server on Modal that can receive, validate, and process webhook events from Slack in real-time.
3. **Creating a One-Way Workflow**: You will implement the core logic to automatically trigger a Manus task whenever your bot is @mentioned in a Slack channel.

By the end of this notebook, you will have a functional, one-way bot. You'll be able to mention it in a Slack channel, and behind the scenes, a Manus task will spring to life

## Setting Up Our Environment

Before we build our server, it's crucial to confirm that our environment is correctly configured with all the necessary API keys.

> Reminder: If you haven't set up your Slack App yet, please follow the Slack Setup guide in the main README.md to get your Bot Token and Signing Secret.

First, let's load our keys from the .env file into our notebook's environment. This cell will read the file and raise an error if any of the required keys are missing.

In [2]:
import os
import requests
from dotenv import load_dotenv
from rich import print

# Load the .env file from the repository root
load_dotenv()

# Fetch credentials from the environment
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
SLACK_SIGNING_SECRET = os.getenv("SLACK_SIGNING_SECRET")

# Validate that all required keys are present
if not all([SLACK_BOT_TOKEN, SLACK_SIGNING_SECRET]):
    raise ValueError(
        "⚠️ One or more required environment variables are missing. "
        "Please ensure MANUS_API_KEY, SLACK_BOT_TOKEN, and SLACK_SIGNING_SECRET are set in your .env file."
    )

print("✓ All API keys and secrets loaded successfully from .env file!")

/Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✓ All API keys and secrets loaded successfully from .env file!

Now let's test our newly added slack keys to make sure they're valid. Slack provides a dedicated `auth.test` API endpoint that is perfect for this. It checks if the token is valid and returns information about the bot and its associated workspace.

In [3]:
# Test the Slack Bot Token
try:
    response = requests.post(
        "https://slack.com/api/auth.test",
        headers={"Authorization": f"Bearer {SLACK_BOT_TOKEN}"},
    )
    response.raise_for_status()
    slack_auth_response = response.json()

    if slack_auth_response.get("ok"):
        team_name = slack_auth_response.get("team")
        user_name = slack_auth_response.get("user")
        print(
            f"[bold green]✓ Slack Bot Token is valid.[/bold green]\n\n   - Authenticated for workspace: [green]'{team_name}'[/green]\n\n   - Authenticated as user: [green]'{user_name}'[/green]"
        )
    else:
        error_message = slack_auth_response.get("error", "unknown_error")
        print(
            f"[bold red]✗ Slack Bot Token is invalid. Slack API returned an error: '{error_message}'[/bold red]"
        )

except requests.exceptions.RequestException as err:
    print(
        f"[bold red]✗ A network error occurred while trying to connect to the Slack API: {err}[/bold red]"
    )

✓ Slack Bot Token is valid.

   - Authenticated for workspace: 'Ivan Personal'

   - Authenticated as user: 'mani'

If all three cells run without errors and show success messages, your environment is perfectly configured, and you're ready to start building the Slack integration

## Building our Interactive Bot

With our credentials verified, it's time to build the core of our application: a server that can listen for Slack events and respond. Our goal for this section is to create the simplest possible interactive bot. 

It will do just one thing: when you @mention it in a channel, it will reply with a predefined message in a thread.

### Defining Our Server

First, we need to write the Python code for our webhook server. 

This server will use FastAPI to define the endpoint and the slack-sdk to help with security.

Let's start by first defining our endpoint as seen below

```py
import modal
import os
import requests
import json
import fastapi
from slack_sdk.signature import SignatureVerifier

# --- Modal App Setup ---
app = modal.App("slack-manus-bridge")
image = modal.Image.debian_slim().pip_install(
    "fastapi[standard]", 
    "requests", 
    "slack-sdk"
)

# Define the secrets we created in the previous step
# We assume a secret named 'slack-credentials' holds our Slack tokens
slack_secret = modal.Secret.from_name("slack-credentials")

@app.function(image=image, secrets=[slack_secret])
@modal.asgi_app()
def fastapi_app():
    web_app = fastapi.FastAPI()

    @web_app.post("/webhooks/slack")
    async def slack_events(request: fastapi.Request):
        """
        Handles all incoming webhook events from Slack.
        """
        # 6. Acknowledge the event to Slack
        return {"status": "ok"}

    return web_app
```

Next, we'll need to run the following commands to store the slack credentials for Modal to use 

```bash
modal secret create slack-credentials SLACK_BOT_TOKEN="<YOUR xoxb- BOT TOKEN>" SLACK_SIGNING_SECRET="<YOUR SIGNING SECRET>"
```

### Deployment

Run the modal serve command in your terminal. This will deploy the code and give you a live, public URL for your webhook.

```bash
modal serve server.py
```

After a moment, Modal will output the URL. Let's copy the base url that they output.

```bash
erve process 78735 terminated
⠸ Running app.../Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
✓ Created objects.
├── 🔨 Created mount /Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/webhook.py
└── 🔨 Created web function fastapi_app => 
    https://ivanleo-manus--slack-manus-bridge-fastapi-app-dev.modal.run
⠸ Running app...
```

In my case here, my base URL is `https://ivanleo-manus--slack-manus-bridge-fastapi-app-dev.modal.run`

In [ ]:
import requests

BASE_URL = "https://ivanleo-manus--slack-manus-bridge-fastapi-app-dev.modal.run"


response = requests.post(
    f"{BASE_URL}/webhooks/slack",
    json={
        "type": "app_mention",
        "user": "U061F7AURT6",
        "text": "<@U061F7AURT6> hello!",
        "ts": "1731887465.000000",
    },
)

print(response.json())

{'status': 'ok'}

Great, we've just implemented our first modal endpoint. In the next section, we'll start implementing our simple webhook so that we can reply to a message with a predefined message

## Understanding Slack Mentions

Now that we've confirmed that our server is live and reachable, let's implement our slack webhook. 

When you `@mention` a bot in Slack, this sends a webhook to your registered endpoint. A webhook here is just a simple HTTP post request that will contain a JSON payload with all the details about the mention.

You don't need to poll their servers, only that you ned to be ready to recieve this event. 

### Registering our Server

From your original `modal serve` output, you have your base URL. This is the base URL plus the endpoint path that we defined earlier. 

In my case here, my full URL is `https://ivanleo-manus--slack-manus-bridge-fastapi-app-dev.modal.run/webhooks/slack`

Before Slack sends any real events, it performs a security handshake. It sends a one-time POST request with a type of "url_verification" and a challenge string. 

Your server must respond with that exact challenge string to prove it's listening. Let's modify our server.py to handle this. We'll also add the crucial step of verifying the request signature to ensure every request is genuinely from Slack.

```py
import modal
import os
import requests
import json
import fastapi
from slack_sdk.signature import SignatureVerifier

# --- Modal App Setup ---
app = modal.App("slack-manus-bridge")
image = modal.Image.debian_slim().pip_install(
    "fastapi[standard]", 
    "requests", 
    "slack-sdk"
)

slack_secret = modal.Secret.from_name("slack-credentials")

@app.function(image=image, secrets=[slack_secret])
@modal.asgi_app()
def fastapi_app():
    web_app = fastapi.FastAPI()

    @web_app.post("/webhooks/slack")
    async def slack_events(request: fastapi.Request):
        """
        Handles all incoming webhook events from Slack.
        """
        # 1. Get the raw request body for signature verification
        body = await request.body()
        headers = request.headers

        # 2. Verify the request is authentically from Slack (critical security step!)
        verifier = SignatureVerifier(os.environ["SLACK_SIGNING_SECRET"])
        if not verifier.is_valid_request(body, headers):
            print("✗ Invalid request signature. Potential security breach!")
            return fastapi.Response(status_code=403)

        # 3. Parse the JSON payload
        payload = json.loads(body)
        print("🔔 Received Slack Event:")
        print(json.dumps(payload, indent=2)) # Pretty print the payload for debugging

        # 4. Handle Slack's one-time URL verification challenge
        if payload.get("type") == "url_verification":
            print("✓ Responding to Slack's URL verification challenge.")
            return {"challenge": payload.get("challenge")}

        # 5. Acknowledge all other events for now
        return {"status": "ok"}

    return web_app
```

Then, navigate to the Event Subscription page in your Slack App Settings 

![](./assets/event_subscription.png)

Then you'll need to add this event URL inside the `Request URL` section. This will trigger a POST request to your webhook endpoint with a similar payload which we'll log out in our Modal logs.

```bash
🔔 Received Slack Event:
{
  "token": "I2cGfd1I4xJbYrcQmdWA6eQQ",
  "challenge": "7r2jFVJAFYH6FEUbenoDJ3qTR5tIwtpiY9XbwGVbWu5UYFBu4Z6N",
  "type": "url_verification"
}
✓ Responding to Slack's URL verification challenge.
```

If you've set this up succesfully, you'll see the following `Verified` status in the same page.

![](./assets/slack_verified_url.png)

### Adding Subscribed Events

Now for the last step, we need to then subscribe to the `read_mention` apo event so that when people mention our bot, we'll recieve an API request.

![](./assets/subscribe_events.png)

Now if we mention our bot in a public channel which it's been added tom we're going to then recieve a mention event like the one below.

```json
{
  "token": "msSmJ25boGYDVco0ygnLwD66",
  "team_id": "T06UWC8HVQ9",
  "api_app_id": "A09U5NYN0BY",
  "event": {
    "type": "app_mention",
    "user": "U06UEQNBFS9",
    "ts": "1763647251.846329",
    "client_msg_id": "9a4f2a63-3d37-46ac-8a62-ee8366e9dc76",
    "text": "<@U09U5R59FDL> hello world!",
    "team": "T06UWC8HVQ9",
    "blocks": [
      {
        "type": "rich_text",
        "block_id": "aLc1U",
        "elements": [
          {
            "type": "rich_text_section",
            "elements": [
              {
                "type": "user",
                "user_id": "U09U5R59FDL"
              },
              {
                "type": "text",
                "text": " hello world!"
              }
            ]
          }
        ]
      }
    ],
    "channel": "C09U5STV0FQ",
    "event_ts": "1763647251.846329"
  },
  "type": "event_callback",
  "event_id": "Ev09ULS1L3Q9",
  "event_time": 1763647251,
  "authorizations": [
    {
      "enterprise_id": null,
      "team_id": "T06UWC8HVQ9",
      "user_id": "U09U5R59FDL",
      "is_bot": true,
      "is_enterprise_install": false
    }
  ],
  "is_ext_shared_channel": false,
  "event_context": "4-eyJldCI6ImFwcF9tZW50aW9uIiwidGlkIjoiVDA2VVdDOEhWUTkiLCJhaWQiOiJBMDlVNU5ZTjBCWSIsImNpZCI6IkMwOVU1U1RWMEZRIn0"
}
```



## Replying in Slack

Now that we're successfully receiving the app_mention event, let's make our bot interactive. We need to do two things:

1. Parse the incoming event to extract the information we need.
2. Use Slack's API to post a reply back to the user.

To reply to a specific message, we need two key pieces of information from the JSON payload we just received:

1. channel: The ID of the channel where the mention occurred. This tells our bot where to send the reply.
2. ts (timestamp): The unique ID of the user's message. By providing this, we tell Slack to post our message as a reply in a thread, which keeps the conversation organized.

We will send our reply using Slack's chat.postMessage API endpoint.

### Sending a Reply

Now let's write a simple function which will send a reply to our original webhook that we recieved above when we had a mention.

In [12]:
from rich import print

def get_slack_client():
    from slack_sdk import WebClient
    client = WebClient(token=os.environ.get("SLACK_BOT_TOKEN"))
    return client

def post_slack_message(channel: str, thread_ts: str, text: str):
    """Posts a message to a specific Slack channel and thread."""
    client = get_slack_client() 
    response = client.chat_postMessage(
        channel=channel,
        thread_ts=thread_ts,
        text=text,
    )
    if not response.get("ok"):
        print(f"Error posting to Slack: {response.get('error')}")
    return response

resp = post_slack_message(
    channel="C09U5STV0FQ", thread_ts="1763647251.846329", text="Hello, World!"
)

print(resp)

{'ok': True, 'channel': 'C09U5STV0FQ', 'ts': '1763648770.123869', 'message': {'user': 'U09U5R59FDL', 'type': 
'message', 'ts': '1763648770.123869', 'bot_id': 'B09U1EUB53M', 'app_id': 'A09U5NYN0BY', 'text': 'Hello, World!', 
'team': 'T06UWC8HVQ9', 'bot_profile': {'id': 'B09U1EUB53M', 'app_id': 'A09U5NYN0BY', 'user_id': 'U09U5R59FDL', 
'name': 'Mani', 'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png', 'image_48': 
'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png', 'image_72': 
'https://a.slack-edge.com/80588/img/plugins/app/service_72.png'}, 'deleted': False, 'updated': 1763646653, 
'team_id': 'T06UWC8HVQ9'}, 'thread_ts': '1763647251.846329', 'parent_user_id': 'U06UEQNBFS9', 'blocks': [{'type': 
'rich_text', 'block_id': 'FWQU', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 
'Hello, World!'}]}]}]}}

If you've copied the right environment variables from your payload that you recieved, you'll see a response of `Hello World!` in your slack channel.

### Sending Files

What happens then if we want to add some files that Manus might have created into this response?

We can use the same `slack_sdk` library to send files to the channel. Let's start by creating a simple dataclass to represent the file.


In [ ]:
from dataclasses import dataclass


@dataclass
class SlackFileAttachment:
    """A simple data class to represent a file to be attached to a Slack message."""

    filename: str
    url: str

def get_slack_client():
    from slack_sdk import WebClient
    client = WebClient(token=os.environ.get("SLACK_BOT_TOKEN"))
    return client

def upload_files(channel:str, files: list[SlackFileAttachment]):
    client = get_slack_client()
    file_ids = []
    for file in files:
        response = client.files_upload_v2(
            channel=channel,
            filename=file.filename,
            content=requests.get(file.url).content,
        )
        if not response.get("ok"):
            print(f"Error uploading {file.filename}: {response.get('error')}")
        else:
            file_ids.append(response.get("file").get("id"))
    return file_ids


upload_files(
    channel="C09U5STV0FQ",
    files=[
        SlackFileAttachment(
            filename="Bershire Hathaway 2025 Investor Letter.pdf",
            url="https://www.berkshirehathaway.com/news/nov1025.pdf",
        )
    ]
)

['F09U657C8TG']

Now, we'll update our original `post_slack_message` function to take in a list of files. 

Let's try it out with just the Bershire Hathaway annual investor letter available at https://www.berkshirehathaway.com/news/nov1025.pdf

In [ ]:
FileID = str

def post_slack_message(
    channel: str, thread_ts: str, text: str, files: List[SlackFileAttachment]
):
    """
    Posts a message with multiple file attachments to a specific Slack channel and thread.

    Args:
        channel (str): The Slack channel ID.
        thread_ts (str): The thread timestamp to reply to.
        text (str): The main message text.
        files (List[SlackFileAttachment]): A list of SlackFileAttachment objects.
    """
    client = get_slack_client()
    attachments: list[FileID] = upload_files(channel, files)
    response = client.chat_postMessage(
        channel=channel,
        thread_ts=thread_ts,
        text=text,
        files=attachments,
    )
    
    if not response.get("ok"):
        print(f"Error posting to Slack: {response.get('error')}")
    return response

resp = post_slack_message(
    channel="C09U5STV0FQ", thread_ts="1763647251.846329", text="Hello, World!", files =[
        SlackFileAttachment(
            filename="Bershire Hathaway 2025 Investor Letter.pdf",
            url="https://www.berkshirehathaway.com/news/nov1025.pdf",
        )
    ]
)
print(resp)

{'ok': True, 'channel': 'C09U5STV0FQ', 'ts': '1763649161.260929', 'message': {'user': 'U09U5R59FDL', 'type': 
'message', 'ts': '1763649161.260929', 'bot_id': 'B09U1EUB53M', 'app_id': 'A09U5NYN0BY', 'text': 'Hello, World!', 
'team': 'T06UWC8HVQ9', 'bot_profile': {'id': 'B09U1EUB53M', 'app_id': 'A09U5NYN0BY', 'user_id': 'U09U5R59FDL', 
'name': 'Mani', 'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png', 'image_48': 
'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png', 'image_72': 
'https://a.slack-edge.com/80588/img/plugins/app/service_72.png'}, 'deleted': False, 'updated': 1763646653, 
'team_id': 'T06UWC8HVQ9'}, 'blocks': [{'type': 'rich_text', 'block_id': '6DanB', 'elements': [{'type': 
'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hello, World!'}]}]}]}}

With this we've implemented a simple helper method that can help us to send messages to slack with files. Let's now use this method to send a message with a file to a specific channel.


## Building our Slack Integration

Now that we are successfully receiving events from Slack, it's time to process them. However, there's a critical rule when working with webhooks: Slack expects a response within three seconds.

If your server takes longer than that to reply, Slack will assume the request failed and will send the same event again. 

Performing time-consuming actions directly in the webhook handler—like calling the Manus API to create a task—will almost certainly exceed this time limit. 

This can lead to duplicate requests and unreliable behavior. 

The solution is to process these requests asynchronously. Our server will perform two steps:

1. **Acknowledge Immediately**: As soon as a request comes in, the webhook will immediately send a 200 OK response back to Slack to let it know the event was received successfully.
2. **Process in the Background**: The actual work of parsing the message and creating a Manus task will be handed off to a background job that can run independently, without making Slack wait.

FastAPI makes this pattern incredibly easy to implement with its BackgroundTasks feature.

### Adding Asynchronous Background Tasks

Let's update our server code. We'll modify the webhook endpoint to be extremely fast and lightweight. Its only jobs are to verify the request's authenticity and schedule the real work to be done in the background.

First, we'll define a function, handle_slack_message, which will contain all the logic for creating a Manus task and posting an initial confirmation message back to Slack.

```py
def handle_slack_message(event: AppMentionEvent) -> None:
    """
    This function runs in the background. It takes the Slack event payload,
    creates a Manus task, and posts a confirmation message back to the original thread.
    """
    channel_id = event["channel"]
    thread_ts = event["ts"]

    # Post a confirmation message back to the user in the same thread.
    post_slack_message(
        channel_id,
        thread_ts,
        f"Task created successfully! You can view its progress here: {task_url}",
        files=[],
    )
```

Next, we update our main webhook handler. 

We'll inject fastapi.BackgroundTasks and use it to schedule our handle_slack_message function whenever a new app_mention event arrives.

```py
@app.function(image=image, secrets=[slack_secret, manus_secret])
@modal.asgi_app()
def fastapi_app():
    web_app = fastapi.FastAPI()

    @web_app.post("/webhooks/slack")
    async def slack_events(
        request: fastapi.Request, background_tasks: fastapi.BackgroundTasks
    ):
        """
        Handles all incoming webhook events from Slack.
        """
        # 1. Verify the request is authentically from Slack
        body = await request.body()
        headers = request.headers
        verifier = SignatureVerifier(os.environ["SLACK_SIGNING_SECRET"])
        if not verifier.is_valid_request(body, headers):
            print("✗ Invalid request signature.")
            return fastapi.Response(status_code=403)

        # 2. Parse the JSON payload
        payload = json.loads(body)

        # 3. Handle Slack's one-time URL verification challenge
        if payload.get("type") == "url_verification":
            return {"challenge": payload.get("challenge")}

        # 4. If it's an app mention, add the handler to background tasks
        if (
            payload.get("type") == "event_callback"
            and payload.get("event", {}).get("type") == "app_mention"
        ):
            # This is the key change: we schedule the work instead of doing it here.
            background_tasks.add_task(handle_slack_message, payload.get("event"))

        # 5. Immediately acknowledge the event to Slack
        return {"status": "ok"}

    return web_app
```

With this new structure, our server is now robust and scalable. It responds to Slack in milliseconds, avoiding timeouts.

Now that we've refactored our code, let's now add in support for creating a Manus Task when a user mentions the bot in a channel.


### Creating our Manus Task

First, let's create a helper function that handles the communication with the Manus API. 

This keeps our code clean and organized. This function will take a text prompt, send it to Manus, and return the response.


In [26]:
def create_manus_task(prompt: str):
    """
    Creates a new task in Manus using the provided prompt.
    """
    api_key = os.environ["MANUS_API_KEY"]
    base_url = "https://api.manus.ai/v1"
    
    response = requests.post(
        f"{base_url}/tasks",
        headers={"API_KEY": api_key},
        json={"prompt": prompt, "agentProfile": "manus-1.5", "taskMode": "chat"},
    )
    
    response.raise_for_status() # Raise an exception for non-200 responses
    return response.json()

To send messages with buttons and other interactive elements, we need to use Slack's Block Kit. 

Block Kit is a UI framework that lets you build rich message layouts using a JSON structure. We'll update our post_slack_message function to support sending these blocks. 

The slack-sdk makes this easy — the `chat_postMessage` method accepts a blocks argument that we can use here.

Let's update our `post_slack_message` to now take in a blocks argument

```py
def post_slack_message(
    channel: str,
    thread_ts: str,
    text: str, # Fallback text for notifications
    blocks: Optional[List[Dict[str, Any]]] = None,
    files: Optional[List[SlackFileAttachment]] = None,
) -> SlackResponse:
    """
    Posts a message with rich blocks and optional file attachments to a 
    specific Slack channel and thread.
    """
    client = get_slack_client()
    attachments: list[FileID] = []
    if files:
        attachments = upload_files(channel, files)

    response: SlackResponse = client.chat_postMessage(
        channel=channel,
        thread_ts=thread_ts,
        text=text,
        blocks=blocks, # Add blocks for rich formatting
        files=attachments,
    )

    if not response.get("ok"):
        print(f"Error posting to Slack: {response.get('error')}")
    return response
```

Now, let's update our background handler, handle_slack_message, to use these new functions. It will call create_manus_task and then construct a Block Kit payload for the response.

```py
def handle_slack_message(event: AppMentionEvent) -> None:
    """
    This function runs in the background. It takes the Slack event payload,
    creates a Manus task, and posts a confirmation message back to the original thread.
    """
    channel_id = event["channel"]
    thread_ts = event["ts"]

    # 1. Clean the prompt text by removing the bot's @mention
    import re
    raw_text = event.get("text", "")
    prompt_text = re.sub(r"^<@[\w\d]+>\s*", "", raw_text)

    # 2. Create the task with Manus
    manus_task = create_manus_task(prompt_text)
    task_id = manus_task.get("task_id")
    task_url = manus_task.get("task_url")
    
    # 3. Build the rich Slack message using Block Kit
    blocks = [
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"Task created successfully! Your request is being processed."
            }
        },
        {
            "type": "actions",
            "elements": [
                {
                    "type": "button",
                    "text": {
                        "type": "plain_text",
                        "text": "View Live Task"
                    },
                    "url": task_url,
                    "action_id": f"view_task_{task_id}" # Unique ID for the action
                }
            ]
        }
    ]

    # 4. Post the confirmation message back to the user
    post_slack_message(
        channel_id,
        thread_ts,
        blocks=blocks
    )
```

Let's now give it a try and see how it works

## Enabling Multi-turn Conversations

Our bot is now functional, but it has a major limitation: every time you mention it, it creates a brand-new Manus task. 

This prevents users from asking follow-up questions or having a continuous conversation within a single Slack thread. We need to store which slack threads are already linked to an active Manus task.

We'll be using two Modal dictionaries here to handle this - one for the thread_ts -> task_id mapping, and one for the task_id -> thread_ts mapping.

Here's how we'll do it. When a new message comes in, our logic will be:

1. Look up the message's thread_ts in our Modal Dict.
2. If a task_id exists: This is a follow-up message. We'll continue the existing Manus task.
3. If no task_id exists: This is the first message in a new conversation. We'll create a new Manus task, post our rich "Task Created" message with the button, and save the new thread_ts -> task_id mapping in the dictionary.

Let's first learn about how Modal's dictionary works and then we'll move into implementing our logic.


### Using Modal's dict

First, we need a way to track which Slack conversations correspond to which Manus tasks. 

Modal's `Dict` is perfect for this - it provides a persistent KV store that can be accessed from anywhere in your server.

First, we'll need to define a new Modal dictionary in our `webhook.py` file

```python
import modal

demo_dict = modal.Dict.from_name("demo-dictionary", create_if_missing=True)

```

Then we'll add two new endpoints to our FastAPI app to interact with the dictionary.


```python
@web_app.post("/set/{key}")
async def set_value(key: str, value: dict):
    """Store a JSON object with the given key."""
    demo_dict[key] = value
    return {"status": "ok", "key": key, "value": value}

@web_app.get("/get/{key}")
async def get_value(key: str):
    """Retrieve a JSON object by key."""
    value = demo_dict.get(key)
    return {"key": key, "value": value}
```

Let's see how we can store some arbitrary task data here for a given taskID which we'll use to reply to our user

In [ ]:
# Example: Store task information when creating a Manus task
task_id = "task_abc123"
task_info = {
    "task_id": task_id,
    "task_url": "https://manus.im/app/task_abc123",
    "slack_channel": "C1234567890",
    "slack_thread_ts": "1678886400.123456",
    "slack_user": "U1234567890",
    "status": "running"
}

# Store the task information
response = requests.post(f"{BASE_URL}/set/{task_id}", json=task_info)
print("Stored:", response.json())

# Later, retrieve the task information
response = requests.get(f"{BASE_URL}/get/{task_id}")
retrieved = response.json()
print("Retrieved:", retrieved)

# Access specific fields
task_data = retrieved["value"]
print(f"Task URL: {task_data['task_url']}")
print(f"Slack Channel: {task_data['slack_channel']}")

Stored:
{
    'status': 'ok',
    'key': 'task_abc123',
    'value': {
        'task_id': 'task_abc123',
        'task_url': 'https://manus.im/app/task_abc123',
        'slack_channel': 'C1234567890',
        'slack_thread_ts': '1678886400.123456',
        'slack_user': 'U1234567890',
        'status': 'running'
    }
}

Retrieved:
{
    'key': 'task_abc123',
    'value': {
        'task_id': 'task_abc123',
        'task_url': 'https://manus.im/app/task_abc123',
        'slack_channel': 'C1234567890',
        'slack_thread_ts': '1678886400.123456',
        'slack_user': 'U1234567890',
        'status': 'running'
    }
}

Task URL: https://manus.im/app/task_abc123

Slack Channel: C1234567890

Now that we've learnt how the Modal dict works, let's integrate it with our slack chatbot and see how we can have a simple multi-turn conversation using the manus slack bot that we've built

### Pushing to the Same Task


First, let's define the modal.Dict at the top of our server file. This dictionary will persist across all function calls.

```py
# Add this line to create our persistent state dictionary
thread_task_map = modal.Dict.from_name(
    "slack-thread-to-task-map", create_if_missing=True
)
```

Next, we need to update create_manus_task to handle both creating new tasks and continuing existing ones. We can do this by adding an optional task_id parameter.

```py
def create_manus_task(prompt: str, task_id: str | None = None) -> dict:
    """
    Creates a new Manus task or continues an existing one.
    """
    api_key = os.environ["MANUS_API_KEY"]
    base_url = "https://api.manus.ai/v1"
    
    payload = {
        "prompt": prompt,
        "agentProfile": "manus-1.5",
        "taskMode": "chat",
    }
    
    # If a task_id is provided, add it to the payload to continue the conversation
    if task_id:
        payload["taskId"] = task_id
        
    response = requests.post(f"{base_url}/tasks", headers={"API_KEY": api_key}, json=payload)
    response.raise_for_status()
    return response.json()
```

Finally, we'll update our main background handler, handle_slack_message, with the core conversational logic.

```py
def handle_slack_message(event: AppMentionEvent) -> None:
    """
    Handles Slack mentions, creating a new Manus task for the first message in a
    thread and continuing the task for any follow-up messages.
    """
    channel_id = event["channel"]
    # Use thread_ts if it exists, otherwise fall back to the message's own ts.
    # This correctly identifies the conversation thread.
    thread_id = event.get("thread_ts", event["ts"])
    
    # Clean the prompt text
    import re
    raw_text = event.get("text", "")
    prompt_text = re.sub(r"^<@[\w\d]+>\s*", "", raw_text)

    client = get_slack_client()
    
    # Check if a task already exists for this thread
    existing_task_id = thread_task_map.get(thread_id)
    
    if existing_task_id:
        # --- This is a follow-up message ---
        print(f"Continuing existing task {existing_task_id} for thread {thread_id}")
        create_manus_task(prompt_text, task_id=existing_task_id)
        
        # Add a reaction to acknowledge the message without cluttering the thread
        client.reactions_add(
            channel=channel_id,
            name="eyes",  # A simple "I'm looking at this" emoji
            timestamp=event["ts"]
        )
    else:
        # --- This is the first message in a new conversation ---
        print(f"Creating new task for thread {thread_id}")
        manus_task = create_manus_task(prompt_text)
        new_task_id = manus_task.get("task_id")
        task_url = manus_task.get("task_url")

        # Save the mapping to our dictionary for future lookups
        thread_task_map[thread_id] = new_task_id
        
        # Build and post the rich "Task Created" message
        blocks = [
            {
                "type": "header",
                "text": { "type": "plain_text", "text": "✅ Task Created Successfully", "emoji": True }
            },
            { "type": "divider" },
            {
                "type": "section",
                "text": { "type": "mrkdwn", "text": f"I've started working on your request!\n\n*Task ID:* `{new_task_id}`" }
            },
            {
                "type": "actions",
                "elements": [
                    {
                        "type": "button",
                        "text": { "type": "plain_text", "text": "🚀 View Live Task", "emoji": True },
                        "url": task_url,
                        "style": "primary"
                    }
                ]
            }
        ]

        post_slack_message(
            channel=channel_id,
            thread_ts=thread_id,
            blocks=blocks,
            text=f"Task created! View progress: {task_url}"
        )
```

However, the loop is only half-complete. Our application can send follow-up messages to Manus, but it doesn't yet receive the results back.

In the next section, we will implement the webhook receiver for Manus events. 

This will allow our bot to post the agent's final response and any generated files back into the correct Slack thread, creating a seamless, two-way conversational experience.

### Closing the Loop

Our bot is now fully conversational—it can create new tasks and handle follow-ups within the same thread. However, the conversation is still one-way. 

Manus completes the work, but the final answer never makes it back to the user in Slack.

First, when we create a new task, we need to save the channel_id and thread_id so that when Manus sends a response, we know where to post it. Let's create a new Modal dictionary to handle this.


```py
# Maps task_id -> { "thread_id": ..., "channel_id": ... }
task_info_map = modal.Dict.from_name(
    "slack-task-to-info-map", create_if_missing=True
)
```

Now, let's update handle_slack_message to save information to both dictionaries when a new task is created.

```py

# ... (after creating the manus_task)
new_task_id = manus_task.get("task_id")
task_url = manus_task.get("task_url")

# Save the mapping in both directions
thread_task_map[thread_id] = new_task_id
task_info_map[new_task_id] = {
    "thread_id": thread_id,
    "channel_id": channel_id,
}
```

Manus responses are often formatted in Markdown, but Slack uses its own version called mrkdwn. 

We'll add a couple of helper functions to convert the response from Manus into rich, well-formatted Slack blocks. This will handle things like headers, dividers, bold text, and links gracefully.

```py
def convert_to_slack_mrkdwn(text: str) -> str:
    """
    Converts standard Markdown to Slack mrkdwn.
    """
    if not text:
        return text
        
    # Bold: **text** -> *text*
    text = re.sub(r'\*\*(.*?)\*\*', r'*\1*', text)
    
    # Links: [text](url) -> <url|text>
    text = re.sub(r'\[([^\]]+)\]\(([^)]+)\)', r'<\2|\1>', text)
    
    return text


def markdown_to_slack_blocks(text: str) -> list[dict]:
    """
    Parses markdown text and converts it into a list of Slack Block Kit blocks.
    Handles headers (###) and dividers (---).
    """
    blocks = []
    lines = text.split('\n')
    current_section_lines = []

    def flush_section():
        if current_section_lines:
            # Join lines and convert standard markdown to slack mrkdwn
            section_text = "\n".join(current_section_lines).strip()
            if section_text:
                # Convert standard markdown to slack mrkdwn
                section_text = convert_to_slack_mrkdwn(section_text)
                blocks.append({
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": section_text
                    }
                })
            current_section_lines.clear()

    for line in lines:
        # Check for headers (e.g. ### Header) - Slack headers are plain text only
        header_match = re.match(r'^(#{1,6})\s+(.+)$', line)
        # Check for dividers (e.g. ---)
        divider_match = re.match(r'^[-*_]{3,}$', line.strip())

        if header_match:
            flush_section()
            header_text = header_match.group(2)
            blocks.append({
                "type": "header",
                "text": {
                    "type": "plain_text",
                    "text": header_text[:3000], # Max length for header
                    "emoji": True
                }
            })
        elif divider_match:
            flush_section()
            blocks.append({"type": "divider"})
        else:
            current_section_lines.append(line)
            
    flush_section()
    return blocks
```

We also need to update our `upload_files` function so that it now takes in a `thread_ts` parameter.

```py
def upload_files(channel: str, files: list[SlackFileAttachment], thread_ts: str) -> list[FileID]:
    client = get_slack_client()
    file_ids = []
    for file in files:
        response: SlackResponse = client.files_upload_v2(
            channel=channel,
            filename=file.filename,
            content=requests.get(file.url).content,
            thread_ts=thread_ts,
        )
        if not response.get("ok"):
            print(f"Error uploading {file.filename}: {response.get('error')}")
        else:
            file_ids.append(response.get("file").get("id"))
    return file_ids
```

Finally, add the new endpoint to your fastapi_app to receive the webhook and trigger the background task.

```py
@web_app.post("/webhooks/manus")
async def manus_events(request: fastapi.Request, background_tasks: fastapi.BackgroundTasks):
    """Handles all incoming webhook events from Manus."""
    payload = await request.json()
    print("Received Manus Event:")
    print(json.dumps(payload, indent=2))
    
    background_tasks.add_task(handle_manus_response, payload)
    return {"status": "ok"}
```

The final step is to tell Manus where to send the events.

1. Make sure your Modal server is running with modal serve server.py.
2. Copy your public URL and append the new endpoint path (e.g., https://<your-modal-url>/webhooks/manus).
3. Go to your Manus account settings, navigate to the "Webhooks" section, and register this new URL.

With this last piece in place, your bot is now complete and fully functional. 

## Conclusion

In this notebook, you moved beyond polling to build an efficient, event-driven solution for monitoring Manus tasks.

Using Modal and FastAPI, you deployed a serverless webhook receiver that instantly processes task events. You learned to securely manage API credentials with Modal Secrets and fetch full conversation histories when tasks complete.

Let's recap the journey so far:

1. Notebook 1: You created your first asynchronous task, learned to poll for its status, and managed a multi-turn conversation by continuing an existing task with new prompts.
2. Notebook 2: You learned the three core methods for attaching files to your tasks: uploading local data directly, referencing public URLs, and embedding in-memory files using Base64 encoding.
3. Notebook 3: You replaced inefficient polling with a professional, event-driven architecture by deploying a serverless webhook receiver with Modal and FastAPI, allowing your application to react instantly when a task is complete.

Now we'll tie everything together by building a Slack bot that  automatically extracts data from receipts, checks them against company policies in Notion, and updates a central expense database—all from a simple file upload in Slack.